In [1]:
# Models:
# Use Regression (? Not sure what methods to try here) to predict visits and spending due to their count nature
# Evaluate using mean absolute error and residual sum of squares
# Fit models on pre-2012 data and generate predictions for 2012+

In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Administrator\Downloads\ml_project_cse204-main\ml_project_cse204-main\Datasets-under-consideration\London\UK_international-visits.csv')
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,year,quarter,market,dur_stay,mode,purpose,area,visits,spend,nights,sample
0,2002,January-March,Belgium,1-3 nights,Air,Holiday,TOTAL ENGLAND,4.431616,1.013484,9.530416,6
1,2002,January-March,Belgium,1-3 nights,Air,Holiday,LONDON,3.572186,0.969138,6.954456,5
2,2002,January-March,Belgium,1-3 nights,Air,Holiday,REST OF ENGLAND,0.859430,0.044346,2.575960,1
3,2002,January-March,Belgium,1-3 nights,Air,Holiday,SOUTHERN ENGLAND,0.859430,0.044346,2.575960,1
4,2002,January-March,Belgium,1-3 nights,Air,Holiday,SOUTH EAST,0.859430,0.044346,2.575960,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549853 entries, 0 to 549852
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   year      549853 non-null  object 
 1   quarter   549853 non-null  object 
 2   market    549853 non-null  object 
 3   dur_stay  549853 non-null  object 
 4   mode      549853 non-null  object 
 5   purpose   549853 non-null  object 
 6   area      549853 non-null  object 
 7   visits    549853 non-null  float64
 8   spend     549853 non-null  float64
 9   nights    549853 non-null  float64
 10  sample    549853 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 46.1+ MB


In [4]:
df.head()

,year,quarter,market,dur_stay,mode,purpose,area,visits,spend,nights,sample
0,2002,January-March,Belgium,1-3 nights,Air,Holiday,TOTAL ENGLAND,4.431616,1.013484,9.530416,6
1,2002,January-March,Belgium,1-3 nights,Air,Holiday,LONDON,3.572186,0.969138,6.954456,5
2,2002,January-March,Belgium,1-3 nights,Air,Holiday,REST OF ENGLAND,0.859430,0.044346,2.575960,1
3,2002,January-March,Belgium,1-3 nights,Air,Holiday,SOUTHERN ENGLAND,0.859430,0.044346,2.575960,1
4,2002,January-March,Belgium,1-3 nights,Air,Holiday,SOUTH EAST,0.859430,0.044346,2.575960,1


In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [6]:
# Set the chunk size for streaming/batch processing
chunk_size = 10000

# Initialize the linear regression models
model_visits = LinearRegression()
model_spend = LinearRegression()

# Initialize variables for accumulating metrics
mae_visits_total = 0.0
rss_visits_total = 0.0
mae_spend_total = 0.0
rss_spend_total = 0.0
num_samples = 0

# Select the features and target variables
target_visits = 'visits'
target_spend = 'spend'

# Separate categorical and numerical features
categorical_features = ['dur_stay','market', 'mode', 'purpose', 'area']
numerical_features = ['nights', 'sample']

# Split the DataFrame into chunks
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

# Iterate over the dataset in chunks
for chunk in chunks:
    # Preprocess categorical features
    chunk_categorical = pd.get_dummies(chunk[categorical_features], columns=categorical_features)

    # Preprocess numerical features
    chunk_numerical = chunk[numerical_features].copy()
    scaler = StandardScaler()
    chunk_numerical[numerical_features] = scaler.fit_transform(chunk_numerical)

    # Concatenate preprocessed categorical and numerical features
    chunk_preprocessed = pd.concat([chunk_categorical, chunk_numerical], axis=1)
    
    # Add target columns to chunk_preprocessed
    chunk_preprocessed[target_visits] = chunk[target_visits]
    chunk_preprocessed[target_spend] = chunk[target_spend]
    
    
    # Split the chunk into features and target variables
    X = chunk_preprocessed.drop([target_visits, target_spend], axis=1)
    y_visits = chunk[target_visits]
    y_spend = chunk[target_spend]

    # Split the chunk into training and testing sets
    X_train, X_test, y_train_visits, y_test_visits = train_test_split(
        X, y_visits, test_size=0.2, random_state=42
    )
    _, _, y_train_spend, y_test_spend = train_test_split(
        X, y_spend, test_size=0.2, random_state=42
    )

    # Fit the linear regression models
    model_visits.fit(X_train, y_train_visits)
    model_spend.fit(X_train, y_train_spend)

    # Make predictions on the testing set
    visits_predictions = model_visits.predict(X_test)
    spend_predictions = model_spend.predict(X_test)

    # Calculate metrics for this chunk
    mae_visits = mean_absolute_error(y_test_visits, visits_predictions)
    rss_visits = mean_squared_error(y_test_visits, visits_predictions) * len(y_test_visits)
    mae_spend = mean_absolute_error(y_test_spend, spend_predictions)
    rss_spend = mean_squared_error(y_test_spend, spend_predictions) * len(y_test_spend)

    # Accumulate metrics for the entire dataset
    mae_visits_total += mae_visits
    rss_visits_total += rss_visits
    mae_spend_total += mae_spend
    rss_spend_total += rss_spend
    num_samples += len(chunk)

# Calculate average metrics for the entire dataset
avg_mae_visits = mae_visits_total / num_samples
avg_rss_visits = rss_visits_total / num_samples
avg_mae_spend = mae_spend_total / num_samples
avg_rss_spend = rss_spend_total / num_samples

print('Visits - Average MAE:', avg_mae_visits)
print('Visits - Average RSS:', avg_rss_visits)
print('Spend - Average MAE:', avg_mae_spend)
print('Spend - Average RSS:', avg_rss_spend)

Visits - Average MAE: 0.00016544509655955802
Visits - Average RSS: 2.5410144178925824
Spend - Average MAE: 0.00012815352523902422
Spend - Average RSS: 2.050035669922573


In [7]:
#  Fit models on pre-2012 data and predictions for 2012+

In [8]:
# replacinng 2019p to 2019
df['year'] = df['year'].replace('2019P', '2019')

In [9]:
# Change data type of Column1 from string to int
df['year'] = df['year'].astype(int)

In [11]:
# Set the target variables
target_visits = 'visits'
target_spend = 'spend'

# Separate categorical and numerical features
categorical_features = ['dur_stay', 'market', 'mode', 'purpose', 'area']
numerical_features = ['year','nights', 'sample']

# Preprocess categorical features
categorical_data = pd.get_dummies(df[categorical_features], columns=categorical_features)

# Preprocess numerical features
numerical_data = df[numerical_features].copy()
scaler = StandardScaler()
numerical_data[numerical_features] = scaler.fit_transform(numerical_data)

# Concatenate preprocessed categorical and numerical features
preprocessed_data = pd.concat([categorical_data, numerical_data], axis=1)

# Add target columns to preprocessed_data
preprocessed_data[target_visits] = df[target_visits]
preprocessed_data[target_spend] = df[target_spend]

# Split the data into features and target variables
X = preprocessed_data.drop([target_visits, target_spend], axis=1)
y_visits = preprocessed_data[target_visits]
y_spend = preprocessed_data[target_spend]

# Filter the data based on the year column
train_mask = df['year'] < 2012
test_mask = df['year'] >= 2012

# Split the data into training and testing sets
X_train = X[train_mask]
X_test = X[test_mask]
y_train_visits = y_visits[train_mask]
y_test_visits = y_visits[test_mask]
y_train_spend = y_spend[train_mask]
y_test_spend = y_spend[test_mask]

# Initialize the linear regression models
model_visits = LinearRegression()
model_spend = LinearRegression()

# Fit the linear regression models
model_visits.fit(X_train, y_train_visits)
model_spend.fit(X_train, y_train_spend)

# Make predictions on the testing set
visits_predictions = model_visits.predict(X_test)
spend_predictions = model_spend.predict(X_test)

# Calculate metrics
mae_visits = mean_absolute_error(y_test_visits, visits_predictions)
rss_visits = mean_squared_error(y_test_visits, visits_predictions) * len(y_test_visits)
mae_spend = mean_absolute_error(y_test_spend, spend_predictions)
rss_spend = mean_squared_error(y_test_spend, spend_predictions) * len(y_test_spend)

print('Visits - MAE:', mae_visits)
print('Visits - RSS:', rss_visits)
print('Spend - MAE:', mae_spend)
print('Spend - RSS:', rss_spend)


Visits - MAE: 2.052457562478504
Visits - RSS: 6306210.798372195
Spend - MAE: 1.5409791505247548
Spend - RSS: 4920885.406873053
